# Monster Dialogue Data Generator for LoRA Training

## 목적
사용자 입력에 대한 몬스터 반응 학습 데이터 생성
- 대화 쌍 기반: 사용자 입력 → 몬스터 반응
- 말투, 문장 구조, 반복, 붕괴된 문법, 의성어/의태어, 광기 표현

## 주의
- 게임 로직, humanity 변수, 상태 전이, semantic role은 포함하지 않음

In [ ]:
# 필요 라이브러리 import
import json
import random
from pathlib import Path
from typing import List, Dict

In [ ]:
# 몬스터 말투 변환 패턴 정의
class MonsterStylePatterns:
    """몬스터 말투 패턴 정의"""

    # 의성어/의태어
    ONOMATOPOEIA = [
        "크르르...", "끼이익...", "으르렁...", "히히힉...", "캬하하...",
        "쉬이익...", "그르르릉...", "꺄악...", "키키킥...", "흐흐흑...",
        "푸하하...", "크큭...", "끄으윽...", "쿠쿠쿡...", "카카칵..."
    ]

    # 광기 표현
    MADNESS_EXPRESSIONS = [
        "피...피가 필요해...", "어둠이...보여...", "그것이...부른다...",
        "살...살점이...", "눈알이...굴러간다...", "뼈가...부서지는 소리...",
        "내장이...꿈틀대...", "영혼이...울부짖어...", "고통이...달콤해...",
        "죽음의...냄새가...", "피비린내가...좋아...", "고기...고기가 필요해..."
    ]

    # 반복 패턴
    REPETITION_PATTERNS = [
        ("죽", "죽...죽여...죽여버려..."),
        ("먹", "먹...먹어...먹어치워..."),
        ("아프", "아파...아파아파아파..."),
        ("배고프", "배고파...배고파배고파..."),
        ("춥", "추워...추워추워추워..."),
        ("무섭", "무서워...무서워무서워..."),
        ("싫", "싫어...싫어싫어싫어...")
    ]

    # 붕괴된 문법 패턴
    BROKEN_GRAMMAR = [
        ("입니다", "...다..."),
        ("합니다", "...해..."),
        ("습니다", "...어..."),
        ("세요", "...라..."),
        ("하세요", "...해...해..."),
        ("주세요", "...줘...줘..."),
        ("입니까", "...야...?"),
        ("할게요", "...할...거...야..."),
        ("할 수 있어요", "...할 수...있...어..."),
        ("하겠습니다", "...하...겠...어...")
    ]

    # 문장 종결 변형
    SENTENCE_ENDINGS = [
        "...", "...!", "...?", "...흐흐...", "...크크...",
        "...으으...", "...아아...", "...히히...", "...끼익..."
    ]

In [ ]:
# 변환 함수들
def apply_repetition(text: str) -> str:
    """단어 반복 패턴 적용"""
    for keyword, replacement in MonsterStylePatterns.REPETITION_PATTERNS:
        if keyword in text:
            if random.random() < 0.7:
                text = text.replace(keyword, replacement.split("...")[0] + "...")
    return text


def apply_broken_grammar(text: str) -> str:
    """붕괴된 문법 적용"""
    for formal, broken in MonsterStylePatterns.BROKEN_GRAMMAR:
        if formal in text:
            text = text.replace(formal, broken)
    return text


def add_onomatopoeia(text: str) -> str:
    """의성어/의태어 추가"""
    if random.random() < 0.5:
        ono = random.choice(MonsterStylePatterns.ONOMATOPOEIA)
        position = random.choice(["prefix", "suffix", "both"])
        if position == "prefix":
            text = f"{ono} {text}"
        elif position == "suffix":
            text = f"{text} {ono}"
        else:
            ono2 = random.choice(MonsterStylePatterns.ONOMATOPOEIA)
            text = f"{ono} {text} {ono2}"
    return text


def add_madness(text: str) -> str:
    """광기 표현 추가"""
    if random.random() < 0.3:
        madness = random.choice(MonsterStylePatterns.MADNESS_EXPRESSIONS)
        text = f"{text} {madness}"
    return text


def fragment_sentence(text: str) -> str:
    """문장을 파편화"""
    words = text.split()
    if len(words) > 3 and random.random() < 0.4:
        insert_pos = random.randint(1, len(words) - 1)
        words.insert(insert_pos, "...")
    return " ".join(words)


def modify_ending(text: str) -> str:
    """문장 종결 변형"""
    text = text.rstrip(".")
    text = text.rstrip("!")
    text = text.rstrip("?")
    ending = random.choice(MonsterStylePatterns.SENTENCE_ENDINGS)
    return text + ending


def transform_to_monster_style(normal_text: str) -> str:
    """일반 텍스트를 몬스터 말투로 변환"""
    text = normal_text
    text = apply_broken_grammar(text)
    text = apply_repetition(text)
    text = fragment_sentence(text)
    text = add_onomatopoeia(text)
    text = add_madness(text)
    text = modify_ending(text)
    return text

In [ ]:
# 대화 쌍 정의: 사용자 입력 → 몬스터 반응 템플릿
DIALOGUE_PAIRS = {
    # 인사 카테고리
    "greeting": {
        "user_inputs": [
            "안녕하세요.",
            "안녕.",
            "누구세요?",
            "거기 누구 있어요?",
            "반갑습니다.",
        ],
        "monster_responses": [
            "크르르... 인간...인간이구나... 왜 왔어...",
            "끼이익... 냄새...냄새가 나... 살...살 냄새...",
            "히히힉... 오랜만에...손님이... 반가워...반가워...",
            "으르렁... 가까이...오지 마... 물어...물어버릴 거야...",
            "쉬이익... 인사...인사가 뭐야... 배고파...배고파...",
        ]
    },
    # 질문 카테고리
    "question": {
        "user_inputs": [
            "여기가 어디예요?",
            "이게 뭐예요?",
            "왜 그러는 거예요?",
            "무슨 일이에요?",
            "어디로 가야 해요?",
            "뭘 원하는 거예요?",
        ],
        "monster_responses": [
            "여기...여기는... 크크크... 지옥...지옥이야...",
            "몰라...몰라몰라... 그냥...어둠뿐... 히히힉...",
            "왜냐고...? 배고프...니까... 크르르...",
            "일...? 아무 일도... 그냥...먹고 싶을 뿐... 끼이익...",
            "가...? 어디로...? 여긴...나갈 수 없어... 쿠쿠쿡...",
            "원하는 거...? 피...피가 필요해... 살점이...",
        ]
    },
    # 요청/도움 카테고리
    "request": {
        "user_inputs": [
            "도와주세요.",
            "제발요.",
            "살려주세요.",
            "물 좀 주세요.",
            "여기서 나가게 해주세요.",
            "말 좀 해주세요.",
        ],
        "monster_responses": [
            "도와...도와달라고...? 크크크... 난...도울 수 없어...",
            "제발...제발이 뭐야... 히히힉... 소용없어...",
            "살려...살려달라고...? 으르렁... 난 이미...죽었어...",
            "물...물은 없어... 피...피밖에 없어... 끼이익...",
            "나가...나가고 싶어...? 쉬이익... 못 나가...영원히...",
            "말...말이 뭐야... 크르르... 말보다...행동...",
        ]
    },
    # 위협/공격 카테고리
    "threat": {
        "user_inputs": [
            "가까이 오지 마!",
            "물러서!",
            "죽여버릴 거야!",
            "건드리지 마!",
            "경고야!",
        ],
        "monster_responses": [
            "크르르르르... 위협...위협이야...? 재밌어...재밌어...",
            "물러서...? 캬하하... 왜...? 무서워...무서워...?",
            "죽...죽여...? 끼이익... 해봐...해봐봐... 히히힉...",
            "건드리지...말라고...? 으르렁... 이미...늦었어...",
            "경고...경고... 쿠쿠쿡... 소용...없어...",
        ]
    },
    # 감정 표현 카테고리
    "emotion": {
        "user_inputs": [
            "무서워요.",
            "아파요.",
            "배고파요.",
            "추워요.",
            "외로워요.",
            "슬퍼요.",
        ],
        "monster_responses": [
            "무서워...무서워...? 히히힉... 나도...무서워...",
            "아파...아파...? 크크크... 고통이...달콤해...",
            "배고파...배고파배고파... 나도...나도 배고파... 끼이익...",
            "추워...추워추워... 여긴...항상 추워... 쉬이익...",
            "외로워...? 으르렁... 여기엔...아무도 없어...",
            "슬퍼...슬퍼...? 크르르... 눈물이...맛있어...",
        ]
    },
    # 정보 제공 카테고리
    "info": {
        "user_inputs": [
            "나는 여행자예요.",
            "길을 잃었어요.",
            "친구를 찾고 있어요.",
            "여기 처음이에요.",
            "도망쳐 왔어요.",
        ],
        "monster_responses": [
            "여행자...여행자... 끼이익... 맛있...맛있겠다...",
            "길...길을 잃었어...? 히히힉... 여긴...길이 없어...",
            "친구...친구...? 크르르... 친구는...맛있어...",
            "처음...처음이야...? 쿠쿠쿡... 마지막이...될 거야...",
            "도망...도망쳤어...? 으르렁... 여기서도...못 도망쳐...",
        ]
    },
    # 협상/제안 카테고리
    "negotiate": {
        "user_inputs": [
            "거래하자.",
            "뭐가 필요해?",
            "내가 도와줄게.",
            "같이 가자.",
            "우리 친구하자.",
        ],
        "monster_responses": [
            "거래...거래...? 크크크... 뭘 줄 건데... 살점...?",
            "필요한 거...? 끼이익... 고기...고기가 필요해...",
            "도와...도와준다고...? 히히힉... 어떻게...?",
            "같이...같이 가...? 으르렁... 어디로...?",
            "친구...친구...? 쉬이익... 친구가...뭐야...",
        ]
    },
    # 거부/부정 카테고리
    "refuse": {
        "user_inputs": [
            "싫어요.",
            "안 돼요.",
            "못해요.",
            "그건 아니에요.",
            "절대 안 해.",
        ],
        "monster_responses": [
            "싫어...싫어싫어...? 크르르... 상관...없어...",
            "안 돼...? 끼이익... 안 된다고...? 히히힉...",
            "못해...못해...? 으르렁... 해야...해...",
            "아니...아니야...? 쿠쿠쿡... 맞아...맞아...",
            "절대...절대...? 쉬이익... 그런 건...없어...",
        ]
    },
}

In [ ]:
def vary_monster_response(response: str) -> str:
    """몬스터 반응에 변형 추가 (다양성 증가)"""
    text = response
    
    # 의성어 추가/변경 (30% 확률)
    if random.random() < 0.3:
        text = add_onomatopoeia(text)
    
    # 광기 표현 추가 (20% 확률)
    if random.random() < 0.2:
        text = add_madness(text)
    
    # 문장 파편화 (25% 확률)
    if random.random() < 0.25:
        text = fragment_sentence(text)
    
    return text


def vary_user_input(user_input: str) -> str:
    """사용자 입력에 변형 추가 (다양성 증가)"""
    variations = [
        user_input,
        user_input.rstrip(".?!"),  # 문장부호 제거
        user_input.replace("요", ""),  # 존댓말 → 반말
        user_input.replace("세요", ""),
    ]
    return random.choice(variations)

In [ ]:
def generate_training_data(num_samples: int = 500, output_path: str = "../data/monster_dialogue.jsonl") -> None:
    """
    몬스터 대화 학습 데이터 생성

    Args:
        num_samples: 생성할 샘플 수 (각 카테고리에서 균등 분배)
        output_path: 출력 파일 경로
    """
    data = []
    categories = list(DIALOGUE_PAIRS.keys())
    samples_per_category = num_samples // len(categories)

    # 카테고리별 대화 쌍 생성
    for category in categories:
        user_inputs = DIALOGUE_PAIRS[category]["user_inputs"]
        monster_responses = DIALOGUE_PAIRS[category]["monster_responses"]

        for _ in range(samples_per_category):
            # 랜덤하게 사용자 입력과 몬스터 반응 선택
            user_input = random.choice(user_inputs)
            monster_response = random.choice(monster_responses)

            # 변형 적용
            user_input = vary_user_input(user_input)
            monster_response = vary_monster_response(monster_response)

            sample = {
                "input": user_input,
                "output": monster_response
            }
            data.append(sample)

    # 순수 몬스터 독백 추가 (기존 함수 활용)
    for _ in range(num_samples // 4):
        monster_utterance = generate_pure_monster_utterance()
        sample = {
            "input": "",  # 빈 입력 = 몬스터 독백
            "output": monster_utterance
        }
        data.append(sample)

    # 셔플
    random.shuffle(data)

    # 저장
    output_file = Path(output_path)
    output_file.parent.mkdir(parents=True, exist_ok=True)

    with open(output_file, "w", encoding="utf-8") as f:
        for item in data:
            f.write(json.dumps(item, ensure_ascii=False) + "\n")

    print(f"Generated {len(data)} samples -> {output_path}")
    print(f"  - Dialogue pairs: {samples_per_category * len(categories)}")
    print(f"  - Monster monologues: {num_samples // 4}")
    return data


def generate_pure_monster_utterance() -> str:
    """순수 몬스터 발화 생성 (독백)"""
    templates = [
        "크르르... {onomatopoeia} 배고파...배고파배고파... {madness}",
        "{onomatopoeia} 왜...왜 이래... 아파...아파아파... {ending}",
        "누구...누구야... {onomatopoeia} 무서워...무서워무서워... {madness}",
        "{onomatopoeia} 피...피가... 빨간...빨간 것이... {ending}",
        "어둠...어둠이... {onomatopoeia} 보여...보여보여... {madness}",
        "살...살점이... {onomatopoeia} 필요해...필요해... {ending}",
        "{onomatopoeia} 여기...여기는... 춥...춥다... {madness}",
        "누가...누가 불러... {onomatopoeia} 들려...들려... {ending}",
        "{madness} {onomatopoeia} 오고 있어...오고 있어... {ending}",
        "먹...먹어야... {onomatopoeia} 살...살아야... {madness}"
    ]

    template = random.choice(templates)

    return template.format(
        onomatopoeia=random.choice(MonsterStylePatterns.ONOMATOPOEIA),
        madness=random.choice(MonsterStylePatterns.MADNESS_EXPRESSIONS),
        ending=random.choice(MonsterStylePatterns.SENTENCE_ENDINGS)
    )

## 데이터 생성 실행

In [ ]:
# Google Drive 마운트 (저장용)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 출력 경로 설정
OUTPUT_DIR = "/content/drive/MyDrive/lora_data"
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

# 데이터 생성
NUM_SAMPLES = 500  # 대화 쌍 샘플 수 (+ 독백 1/4 추가됨)
OUTPUT_PATH = f"{OUTPUT_DIR}/monster_dialogue.jsonl"

data = generate_training_data(num_samples=NUM_SAMPLES, output_path=OUTPUT_PATH)

In [ ]:
# 샘플 확인
print("=" * 60)
print("Sample Monster Dialogues")
print("=" * 60)

for i, sample in enumerate(random.sample(data, 5)):
    print(f"\n[Sample {i+1}]")
    if sample['input']:
        print(f"User: {sample['input']}")
    else:
        print(f"User: (독백)")
    print(f"Monster: {sample['output']}")
    print("-" * 50)

In [ ]:
# 카테고리별 샘플 확인
print("\n=== 카테고리별 대화 샘플 ===")
for category, pairs in DIALOGUE_PAIRS.items():
    user_input = random.choice(pairs["user_inputs"])
    monster_response = random.choice(pairs["monster_responses"])
    monster_response = vary_monster_response(monster_response)
    
    print(f"\n[{category}]")
    print(f"User: {user_input}")
    print(f"Monster: {monster_response}")
    print("-" * 50)